# Web Services Data Exercise

This exercise is a script for downloading either miniSEED or RINEX data from SAGE and GAGE web services. All the functions are covered in the previous `4_data_access.ipynb` notebook. The download function accomodates either a SAGE web service request or GAGE web service request. Fill in the blanks to complete the script.

Download earthquake event data by the query parameters for the SAGE web service. You can use the example in the previous notebook or use another event and station. 

Download a RINEX file by providing a date, a station, and the type of compression. You can use the example in the previous notebook or use another event and station.

If the script is correctly completed you will have a new directory calle `/data` containig a miniSEED and RINEX file.

## Instructions
This exercise is a script for accessing EarthScope data using web services.

Create a notebook and title it web_services_data_exercise.ipynb.
Create three code cells and copy the exercise code into the respective code cell.
Fill in the blanks as required.

In [ ]:
# ---------- Fill in the package imports for this script ----------
import __BLANK1__  
from datetime import datetime
from pathlib import Path
import __BLANK2__

# ---------- Create an EarthScope client to get token ----------
client = __BLANK3__


# Fill this with the service endpoint for SAGE data
SAGE_URL = __BLANK4__
# Fill this with the real base path for GAGE data
GAGE_URL = __BLANK5__ 

DATA_DIR = Path("./data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# ---------- Get an EarthScope token ----------
def get_token():
    """
    Use the SDK to refresh (if needed) and return an access token.
    """
    # Refresh if necessary
    __BLANK6__

    # Retrieve the token string
    token = __BLANK7__
    return token


# ---------- Helper: common auth header ----------
def auth_header(token):
    """
    Helper function to create header with token
    """
    # Use a standard Bearer token header (lowercase 'authorization' is fine)
    return __BLANK8__


# ---------- Download a file from EarthScope's web services ----------
# --------------------------------------------------------------------
def download_data(url, data_directory, params={}):  # params is an optional query parameter
    """
    Sends GET with query parameters to the SAGE web service and saves the response 
    body to a file. Expected params include: net, sta, loc, cha, start (ISO), end (ISO), etc.
    Sends GET request to GAGE web service by construcint the URL to the RINEX file.
    """
    # get authorization token
    token = get_token()

    # create the authorization header 
    header = auth_header(token) 


    # This creates a file name for either a miniSEED or RINEX file
    # depending upon if the params dictionary is empty. The bool comparator
    # returns True if the params dictionary is not empty 
    if bool(params): 
        # Parse the start datetime to build filename parts
        start_dt = datetime.strptime(params["start"], "%Y-%m-%dT%H:%M:%S")
        year = str(start_dt.year) # Day-of-year (001-366) for file naming
        doy =  "{:03d}".format(__BLANK10__)  # hint: see how year is extracted from the start
        
        # create a file name for a miniSEED
        file_name = ".".join([
            params["sta"],
            params["net"],
            params["loc"],
            params["cha"],
            year,
            doy,
            "mseed"
        ])
    else: # create a file name for a RINEX file if the params dictionary is empty
        file_name = Path(url).name

    # create a variable with the path to the data directory and file
    out_path = Path(Path(data_directory) / __BLANK11__)

    # Make the request to the web service with params and bearer auth
    r = requests.__BLANK12__(
        url,
        params=__BLANK13__,
        headers=__BLANK14__,
        stream=True
    ) 

    if r.status_code == requests.codes.__BLANK15__:
        with open(out_path, "wb") as f:
            for data in r:
                f.write(data)
    else:
        #problem occured
        print(f"failure: {r.status_code}, {r.reason}")
        return None


# ---------- Creates URL to download data from the EarthScope GAGE web service ----------
# ---------------------------------------------------------------------------------------
def create_url(year, day, station, compression):
    """
    Construct a URL to a file in an archive given year, day-of-year, station, etc.
    Example output:
      {BASE_URL}{year}/{DOY}/{/STATIONDOY0.YY}{compression}
    """
    doy =  "{:03d}".__BLANK16__(day)
    two_digit_year = __BLANK17__  # Hint: use the string slice function in the previous notebook

    file_path = "/".join([str(year), doy])
    file_name = "".join(["/", station, doy, "0.", two_digit_year, compression])
    url = "".join([GAGE_URL, file_path, file_name])
    return url


In [ ]:
# Download a miniSEED file for an event
# 
params = {"net" : 'IU',
          "sta" : 'ANMO',
          "loc" : '00',
          "cha" : 'BHZ',
          "start": '2010-02-27T06:30:00',
          "end": '2010-02-27T10:30:00'}

# Try the parameterized request
download_data(__BLANK18__, DATA_DIR, __BLANK19_)

In [ ]:
# Download a RINEX file
#
year = 2025
day = 1
station = 'p034'
doy = '%03d'.format(day)
compression = 'd.Z'  # or ".Z" / "" depending on the archive
url = create_url(year, __BLANK20__, station, compression)
download_data(url, __BLANK21__)

```{admonition} Click to see answers
:class: dropdown
__BLANK1__ → import requests

__BLANK2__ → from earthscope_sdk import EarthScopeClient

__BLANK3__ → client = EarthScopeClient()

__BLANK4__ → a valid string endpoint for the parameterized API, e.g. "https://example.earthscope.org/api/waveforms"

__BLANK5__ → a valid base URL for direct files, e.g. "https://example.earthscope.org/gage/archive/"

__BLANK6__ → client.ctx.auth_flow.refresh_if_necessary()

__BLANK7__ → client.ctx.auth_flow.access_token

__BLANK8__ → {"authorization": f"Bearer {token}"}

__BLANK9__ → requests

__BLANK10__ → "{:03d}".format(day)

__BLANK11__ → file_name

__BLANK12__ → requests.get(

__BLANK13__ → params

__BLANK14__ → header

__BLANK15__ → requests.codes.ok

__BLANK16__ → "{:03d}".format(day)

__BLANK17__ → str(year)[2:]

__BLANK18__ → url

__BLANK19__ → params

__BLANK20__ → day

__BLANK21__ → DATA_DIR
```